In [ ]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [ ]:
# Tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace("?",'')
  text = text.replace("'","")
  return text.split()

In [ ]:
tokenize("Who wrote 'To Kill a Mockingbird'?")

['who', 'wrote', 'to', 'kill', 'a', 'mockingbird']

In [ ]:
vocabulary = {'<UNK>':0}

def all_vocabulary(row):
  question_tokenize = tokenize(row['question'])
  answer_tokenize = tokenize(row['answer'])
  merged_tokenize = question_tokenize + answer_tokenize

  for token in merged_tokenize:
    if token not in vocabulary:
      vocabulary[token] = len(vocabulary)

In [ ]:
df.apply(all_vocabulary,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [ ]:
vocabulary

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [ ]:
def text_to_numeric(text,vocab):
  numeric_text =[]
  token_text = tokenize(text)

  for token in token_text:
    if token in vocab:
      numeric_text.append(vocab[token])
    else:
      numeric_text.append(vocab['<UNK>'])
  return numeric_text


In [ ]:
numeric=text_to_numeric('who is Harshith',vocabulary)
print(numeric)

[10, 2, 0]


In [ ]:
df['question'].iloc[1]

'What is the capital of Germany?'

In [ ]:
df.iloc[1]['question']

'What is the capital of Germany?'

In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader

class Mydataset(Dataset):

  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,index): # we want to give the output in neumeric formate
    tokenize_question=text_to_numeric(self.df['question'].iloc[index],self.vocab)
    tokenize_answer=text_to_numeric(self.df['answer'].iloc[index],self.vocab)
# Pytorch wants to return in tensor formate
    return torch.tensor(tokenize_question),torch.tensor(tokenize_answer)

In [ ]:
data = Mydataset(df,vocabulary)

In [ ]:
data_loader = DataLoader(data,batch_size=1,shuffle=True)

In [ ]:
import torch.nn as nn

class Mymodel(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn = nn.RNN(50,64,batch_first=True)
    self.fc = nn.Linear(64,vocab_size)

  def forward(self,feature):
    output = self.embedding(feature)
    hidden_output,final_output = self.rnn(output)
    output = self.fc(final_output.squeeze(0))
    return output

In [ ]:
lr_rate=0.0001
epochs=100
model =Mymodel(len(vocabulary))
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr_rate)

In [ ]:
# Training

for epoch in range(epochs):
  total_loss=0
  for question,answer in data_loader:

    optimizer.zero_grad()

    pred = model(question)
    print(pred.shape)

    loss = loss_func(pred,answer[0])

    loss.backward()

    optimizer.step()

    total_loss+=loss.item()

  avg_loss = total_loss/len(data_loader)
  print(f"{epoch+1}:{avg_loss}")


Streaming output truncated to the last 5000 lines.
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torch.Size([1, 324])
torc

In [ ]:
def prediction(model,question,threshold):
  neumeric_question = text_to_numeric(question,vocabulary)
  tensor_question = torch.tensor(neumeric_question).unsqueeze(0)
  output = model(tensor_question)
  probs= torch.nn.functional.softmax(output,dim=1)
  value,index = torch.max(probs,dim=1)

  if value<threshold:
    print("I don't know")
  else:
    print(list(vocabulary.keys())[index])


In [ ]:
prediction(model,"Who wrote 'To Kill a Mockingbird'?",0.5)

harper-lee


In [ ]:
prediction(model,"What is the boiling point of water in Celsius?",0.5)

100


In [ ]:
prediction(model,'Who is pytorch',0.5)

I don't know
